# Task7 :  Simple moving average

Inside Stock, Let's create a method that calculates the simple moving average for the given stock

Be careful: Expensive computation requires optimization

In [1]:
import sys  
sys.path.insert(0, '/root/work/functions.py')
from functions import *
import pandas as pd

In [2]:
data=create_dataframe('SLAFIN.aspx')

In [3]:
data

,date,closing,adjusted,evolution,quantity,volume
0,2020-01-04,631.7,631.7,0.00,0.0,0.0
1,2020-01-06,515.0,515.0,0.00,0.0,0.0
2,2020-01-07,527.5,527.5,-1.22,29.0,15336.5
3,2020-01-09,481.0,481.0,0.21,5.0,2405.0
4,2020-01-10,480.0,480.0,0.78,7.0,3342.3
...,...,...,...,...,...,...
727,2022-12-05,731.0,731.0,5.94,948.0,674080.0
728,2022-12-07,615.0,615.0,0.00,0.0,0.0
729,2022-12-08,622.0,622.0,0.00,0.0,0.0
730,2022-12-09,664.0,664.0,0.00,0.0,0.0


# Creating the function:

In [4]:
def simple_moving_average(period):

    sma = []
    price=[]
    for i in range(len(data) - period+1):
        sma.append(sum(data['closing'][i:i+period])/period)
        price.append(data['closing'][i])
    d={'closing':price,'moving_avg':sma}
    df= pd.DataFrame(d)
    return df

In [5]:
simple_moving_average(10)

,closing,moving_avg
0,631.7,629.72
1,515.0,644.55
2,527.5,671.05
3,481.0,696.30
4,480.0,728.20
...,...,...
718,550.0,595.61
719,550.0,602.11
720,543.0,609.31
721,550.0,621.41


# The improved function 

Instead of using a for loop to iterate over the range of data points and calculate the moving average, 

you can use the rolling method provided by pandas to calculate the moving average directly. This will be much faster than iterating over the data manually.

Also, instead of creating two separate lists for the closing price and the moving average, you can create a new DataFrame directly, with the closing price and the moving average as columns.

In [6]:
def simple_moving_average(data, period):
    df = data.copy()
    df["moving_avg"] = df["closing"].rolling(window=period).mean()
    return df

In [7]:
simple_moving_average(data,10)

,date,closing,adjusted,evolution,quantity,volume,moving_avg
0,2020-01-04,631.7,631.7,0.00,0.0,0.0,NaN
1,2020-01-06,515.0,515.0,0.00,0.0,0.0,NaN
2,2020-01-07,527.5,527.5,-1.22,29.0,15336.5,NaN
3,2020-01-09,481.0,481.0,0.21,5.0,2405.0,NaN
4,2020-01-10,480.0,480.0,0.78,7.0,3342.3,NaN
...,...,...,...,...,...,...,...
727,2022-12-05,731.0,731.0,5.94,948.0,674080.0,595.61
728,2022-12-07,615.0,615.0,0.00,0.0,0.0,602.11
729,2022-12-08,622.0,622.0,0.00,0.0,0.0,609.31
730,2022-12-09,664.0,664.0,0.00,0.0,0.0,621.41


# Adding it as a method to stock class

In [10]:
class Stock:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    def __init__(self, name):
        self.name = name
        self.data = create_dataframe(name+'.aspx')
    
    #methods to get the max and min of different columns
    def return_max(self):
        return max(self.data['closing'])
    def return_min(self):
        return min(self.data['closing'])
    def volume_max(self):
        return max(self.data['volume'])
    def quantity_max(self):
        return max(self.data['quantity'])

        
   
    def plot_graph(self,Type):

        # Create figure with secondary y-axis
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        # Add traces(the price graph)
        fig.add_trace(go.Scatter(x=data['date'], y=data[Type], name=Type),secondary_y=False,)
        
        #add the volume graph
        fig.add_trace(
            go.Bar(x=data['date'], y=data['volume'],name='volume'),
            secondary_y=True,
        )
        #change the size of the figure
        fig.update_layout(
            autosize=False,
            width=900,
            height=500,paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)')
        # Set x-axis title
        fig.update_xaxes(title_text="Date")

        # Set y-axes titles
        fig.update_yaxes(title_text="<b>Price</b>", secondary_y=False,autorange=False)
        fig.update_yaxes(title_text="<b>Volume</b>", secondary_y=True)
        fig.update_yaxes(range=[50,1600], secondary_y=False)
        fig.show()
        #becuase plotly doesn't show in github
        fig.write_image("img.png")
    

    
    #the function that calculate the momentum indcator
    def momentum_indicator(self, column, time_steps):
        subdata = self.data[['date', column]].copy()
        for step in time_steps:
            momentum_col = f"{column}_momentum_{step}"
            subdata[momentum_col] = subdata[column].shift(step) - subdata[column]
        return subdata


#the function that calculate the
    def simple_moving_average(self, period):
        df = self.data.copy()
        df["moving_avg"] = df["closing"].rolling(window=period).mean()
        return df


In [11]:
s1 = Stock('EQDOM')
s1.simple_moving_average(3)

,date,closing,adjusted,evolution,quantity,volume,moving_avg
0,2020-01-04,1190.0,1190.0,0.00,1781.0,2119390.0,NaN
1,2020-01-06,1020.0,1020.0,0.00,0.0,0.0,NaN
2,2020-01-07,1098.0,1098.0,-0.18,22.0,23724.0,1102.666667
3,2020-01-09,1020.0,1020.0,0.99,112.0,114265.0,1046.000000
4,2020-01-10,992.0,992.0,0.00,0.0,0.0,1036.666667
...,...,...,...,...,...,...,...
727,2022-12-05,1200.0,1200.0,0.00,8.0,9600.0,1339.333333
728,2022-12-07,1057.0,1057.0,0.00,0.0,0.0,1218.333333
729,2022-12-08,1174.0,1174.0,5.77,1.0,1174.0,1143.666667
730,2022-12-09,1200.0,1200.0,0.00,0.0,0.0,1143.666667


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8eeb6fb9-2d6b-464e-b219-535fd1ffa708' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>